In [ ]:
# ubuntu version
!lsb_release -a

In [ ]:
# check which version of CUDA is installed
!nvcc --version

In [ ]:
# check CUDA location
!which nvcc

In [ ]:
# install CUDA 11.6.2 https://stackoverflow.com/questions/50560395/how-to-install-cuda-in-google-colab-gpus
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
!mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.6.2/local_installers/cuda-repo-ubuntu2004-11-6-local_11.6.2-510.47.03-1_amd64.deb
!dpkg -i cuda-repo-ubuntu2004-11-6-local_11.6.2-510.47.0-1_amd64.deb
!apt-key add /var/cuda-repo-ubuntu2004-11-6-local/7fa2af80.pub
!apt-get update
!apt-get -y install cuda-11-6

In [ ]:
# check if installed
!ls -la /usr/local

In [6]:
# symlink to be the current CUDA
!rm /usr/local/cuda
!ln -s /usr/local/cuda-11.6 /usr/local/cuda

In [ ]:
!nvcc --version

In [ ]:
# check if conda is installed
!conda --version

In [ ]:
# install conda
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
# check installation
!conda --version

In [ ]:
# check installation location
!which conda

In [ ]:
# create earthformer environment
!conda init bash
!conda create -n earthformer python=3.9

In [ ]:
%%shell
eval "$(conda shell.bash hook)" # copy conda command to shell
conda activate earthformer
conda info
nvcc --version
# install dependencies
pip install --upgrade pip
python3 -m pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 -f https://download.pytorch.org/whl/torch_stable.html
python3 -m pip install pytorch_lightning>=1.6.4
python3 -m pip install xarray netcdf4 opencv-python
git clone https://github.com/amazon-science/earth-forecasting-transformer.git
cd earth-forecasting-transformer
python3 -m pip install -U -e . --no-build-isolation
python3 -m pip install setuptools
python3 -m pip install packaging
pip install --upgrade setuptools wheel Cmake numpy numba ninja
env CUDA_HOME=/usr/local/cuda-11.6
pip install -v --disable-pip-version-check --no-cache-dir pytorch-extension git+https://github.com/NVIDIA/apex.git

In [ ]:
%%shell
# install aws cli to check the size of the sevir dataset
eval "$(conda shell.bash hook)" # copy conda command to shell
conda activate earthformer
conda info
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip awscliv2.zip
sudo ./aws/install

In [ ]:
%%shell
# checking the aws cli installation and the size of the sevir dataset
eval "$(conda shell.bash hook)" # copy conda command to shell
conda activate earthformer
aws --version
aws s3 ls --no-sign-request s3://sevir/data/vil --recursive --summarize --human-readable

In [16]:
# %%shell
# # downloading sevir
# eval "$(conda shell.bash hook)" # copy conda command to shell
# conda activate earthformer
# cd earth-forecasting-transformer
# rm -d -r ./datasets/sevir
# python3 ./scripts/datasets/sevir/download_sevir.py --dataset sevir

In [22]:
%cd earth-forecasting-transformer

/content/earth-forecasting-transformer/earth-forecasting-transformer


In [25]:
!python3 -m pip install -q "pytorch_lightning>=1.6.4,<1.8.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.1/708.1 kB 9.5 MB/s eta 0:00:00


In [26]:

import warnings
import os
from omegaconf import OmegaConf
import pytest
import numpy as np
import torch
from torch.nn import functional as F
import torchmetrics
from einops import rearrange
from src.earthformer.config import cfg
from src.earthformer.utils.checkpoint import s3_download_pretrained_ckpt
from src.earthformer.utils.layout import layout_to_in_out_slice
from src.earthformer.utils.utils import download
from src.earthformer.cuboid_transformer.cuboid_transformer import CuboidTransformerModel
from src.earthformer.cuboid_transformer.cuboid_transformer_unet_dec import CuboidTransformerAuxModel

ImportError: ignored